# Module 22: Monitoring - Keeping Models Healthy

**Goal:** Learn how to detect model degradation, set up alerts, and know when to retrain.

**Prerequisites:** Module 21 (MLOps)

**Expected Runtime:** ~25 minutes

**Outputs:**
- Implemented drift detection
- Built alerting system
- Created monitoring dashboard

---

## Setup

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy import stats
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, roc_auc_score
from datetime import datetime, timedelta
from typing import Dict, List, Tuple, Optional
from dataclasses import dataclass
import warnings
warnings.filterwarnings('ignore')

np.random.seed(42)
plt.style.use('seaborn-v0_8-whitegrid')

## Part 1: Generate Data with Drift

In [ ]:
def generate_data(n_samples: int, drift_factor: float = 0.0, concept_drift: float = 0.0):
    """
    Generate data with optional drift.
    
    Args:
        n_samples: Number of samples
        drift_factor: Amount of data drift (0-1)
        concept_drift: Amount of concept drift (0-1)
    """
    # Base features
    tenure = np.random.exponential(20, n_samples)
    spend = np.random.normal(100, 30, n_samples)
    support_tickets = np.random.poisson(2, n_samples)
    engagement = np.random.beta(5, 2, n_samples) * 100
    
    # Apply data drift (shift distributions)
    if drift_factor > 0:
        tenure = tenure * (1 + drift_factor * 0.5)  # Older customers
        spend = spend * (1 + drift_factor * 0.3)     # Higher spend
        engagement = engagement * (1 - drift_factor * 0.2)  # Lower engagement
    
    # Base churn probability
    churn_prob = 0.2 + 0.2 * (support_tickets > 3) - 0.1 * (engagement > 70)
    
    # Apply concept drift (change relationship)
    if concept_drift > 0:
        # The relationship changes - support tickets become less predictive
        churn_prob = 0.2 + 0.1 * (support_tickets > 3) - 0.05 * (engagement > 70)
        # New factor becomes important
        churn_prob += concept_drift * 0.2 * (spend < 80)
    
    churn_prob = np.clip(churn_prob, 0.05, 0.95)
    churned = (np.random.random(n_samples) < churn_prob).astype(int)
    
    return pd.DataFrame({
        'tenure': tenure,
        'spend': spend,
        'support_tickets': support_tickets,
        'engagement': engagement,
        'churned': churned
    })

# Generate training data (no drift)
train_data = generate_data(2000, drift_factor=0, concept_drift=0)
print(f"Training data: {len(train_data)} rows, churn rate: {train_data['churned'].mean():.1%}")

# Generate production data with drift
prod_data_clean = generate_data(500, drift_factor=0, concept_drift=0)
prod_data_drift = generate_data(500, drift_factor=0.5, concept_drift=0)
prod_data_concept = generate_data(500, drift_factor=0, concept_drift=0.5)
prod_data_both = generate_data(500, drift_factor=0.5, concept_drift=0.5)

print("Production datasets generated with various drift levels.")

## Part 2: Train Baseline Model

In [ ]:
# Prepare data
features = ['tenure', 'spend', 'support_tickets', 'engagement']
X_train = train_data[features]
y_train = train_data['churned']

# Train model
model = GradientBoostingClassifier(n_estimators=100, max_depth=4, random_state=42)
model.fit(X_train, y_train)

# Baseline metrics
train_preds = model.predict(X_train)
train_probs = model.predict_proba(X_train)[:, 1]

baseline_metrics = {
    'accuracy': accuracy_score(y_train, train_preds),
    'auc': roc_auc_score(y_train, train_probs),
    'pred_mean': train_probs.mean(),
    'pred_std': train_probs.std()
}

print("=== Baseline Metrics ===")
for k, v in baseline_metrics.items():
    print(f"  {k}: {v:.3f}")

## Part 3: Drift Detection

In [ ]:
def calculate_psi(expected: np.ndarray, actual: np.ndarray, buckets: int = 10) -> float:
    """Calculate Population Stability Index."""
    # Create buckets based on expected distribution
    bins = np.percentile(expected, np.linspace(0, 100, buckets + 1))
    bins[0] = -np.inf
    bins[-1] = np.inf
    
    # Calculate proportions
    expected_counts = np.histogram(expected, bins)[0] / len(expected)
    actual_counts = np.histogram(actual, bins)[0] / len(actual)
    
    # Avoid division by zero
    expected_counts = np.clip(expected_counts, 0.0001, None)
    actual_counts = np.clip(actual_counts, 0.0001, None)
    
    # Calculate PSI
    psi = np.sum((actual_counts - expected_counts) * np.log(actual_counts / expected_counts))
    return psi

def detect_drift(reference: pd.DataFrame, current: pd.DataFrame, features: List[str]) -> Dict:
    """Detect drift across features."""
    results = {}
    
    for feature in features:
        # KS test
        ks_stat, ks_pvalue = stats.ks_2samp(reference[feature], current[feature])
        
        # PSI
        psi = calculate_psi(reference[feature].values, current[feature].values)
        
        results[feature] = {
            'ks_statistic': ks_stat,
            'ks_pvalue': ks_pvalue,
            'psi': psi,
            'drift_detected': psi > 0.1 or ks_pvalue < 0.05
        }
    
    return results

# Test drift detection
print("=== Drift Detection Results ===")

scenarios = [
    ("No Drift", prod_data_clean),
    ("Data Drift", prod_data_drift),
    ("Concept Drift", prod_data_concept),
    ("Both Drifts", prod_data_both)
]

for name, data in scenarios:
    drift_results = detect_drift(train_data, data, features)
    drifted_features = [f for f, r in drift_results.items() if r['drift_detected']]
    avg_psi = np.mean([r['psi'] for r in drift_results.values()])
    print(f"\n{name}:")
    print(f"  Drifted features: {drifted_features if drifted_features else 'None'}")
    print(f"  Average PSI: {avg_psi:.3f}")

## Part 4: Performance Monitoring

In [ ]:
def monitor_performance(model, data: pd.DataFrame, features: List[str], baseline: Dict) -> Dict:
    """Monitor model performance on new data."""
    X = data[features]
    y = data['churned']
    
    preds = model.predict(X)
    probs = model.predict_proba(X)[:, 1]
    
    current_metrics = {
        'accuracy': accuracy_score(y, preds),
        'auc': roc_auc_score(y, probs),
        'pred_mean': probs.mean(),
        'pred_std': probs.std()
    }
    
    # Compare to baseline
    degradation = {
        'accuracy_drop': baseline['accuracy'] - current_metrics['accuracy'],
        'auc_drop': baseline['auc'] - current_metrics['auc'],
        'pred_mean_shift': abs(current_metrics['pred_mean'] - baseline['pred_mean'])
    }
    
    return {
        'current': current_metrics,
        'degradation': degradation,
        'alert': degradation['accuracy_drop'] > 0.05 or degradation['auc_drop'] > 0.05
    }

# Monitor across scenarios
print("=== Performance Monitoring ===")

for name, data in scenarios:
    perf = monitor_performance(model, data, features, baseline_metrics)
    print(f"\n{name}:")
    print(f"  Current Accuracy: {perf['current']['accuracy']:.3f}")
    print(f"  Accuracy Drop: {perf['degradation']['accuracy_drop']:.3f}")
    print(f"  Alert: {'🚨 YES' if perf['alert'] else '✓ No'}")

## Part 5: Alerting System

In [ ]:
@dataclass
class Alert:
    level: str  # info, warning, critical
    metric: str
    message: str
    value: float
    threshold: float
    timestamp: str

class AlertingSystem:
    """Simple alerting system for model monitoring."""
    
    def __init__(self, config: Dict):
        self.config = config
        self.alerts: List[Alert] = []
    
    def check(self, metrics: Dict) -> List[Alert]:
        """Check metrics against thresholds."""
        new_alerts = []
        timestamp = datetime.now().isoformat()
        
        # Check accuracy
        if metrics.get('accuracy_drop', 0) > self.config['critical']['accuracy_drop']:
            new_alerts.append(Alert(
                level='critical',
                metric='accuracy',
                message=f"Accuracy dropped by {metrics['accuracy_drop']:.1%}",
                value=metrics['accuracy_drop'],
                threshold=self.config['critical']['accuracy_drop'],
                timestamp=timestamp
            ))
        elif metrics.get('accuracy_drop', 0) > self.config['warning']['accuracy_drop']:
            new_alerts.append(Alert(
                level='warning',
                metric='accuracy',
                message=f"Accuracy dropped by {metrics['accuracy_drop']:.1%}",
                value=metrics['accuracy_drop'],
                threshold=self.config['warning']['accuracy_drop'],
                timestamp=timestamp
            ))
        
        # Check PSI
        if metrics.get('psi', 0) > self.config['critical']['psi']:
            new_alerts.append(Alert(
                level='critical',
                metric='psi',
                message=f"Significant drift detected (PSI: {metrics['psi']:.2f})",
                value=metrics['psi'],
                threshold=self.config['critical']['psi'],
                timestamp=timestamp
            ))
        elif metrics.get('psi', 0) > self.config['warning']['psi']:
            new_alerts.append(Alert(
                level='warning',
                metric='psi',
                message=f"Moderate drift detected (PSI: {metrics['psi']:.2f})",
                value=metrics['psi'],
                threshold=self.config['warning']['psi'],
                timestamp=timestamp
            ))
        
        self.alerts.extend(new_alerts)
        return new_alerts

# Configure alerting
alert_config = {
    'warning': {'accuracy_drop': 0.03, 'psi': 0.1},
    'critical': {'accuracy_drop': 0.05, 'psi': 0.25}
}

alerting = AlertingSystem(alert_config)

# Test alerting
print("=== Alerting System ===")

for name, data in scenarios:
    perf = monitor_performance(model, data, features, baseline_metrics)
    drift = detect_drift(train_data, data, features)
    avg_psi = np.mean([r['psi'] for r in drift.values()])
    
    metrics = {
        'accuracy_drop': perf['degradation']['accuracy_drop'],
        'psi': avg_psi
    }
    
    alerts = alerting.check(metrics)
    
    print(f"\n{name}:")
    if alerts:
        for alert in alerts:
            icon = '🚨' if alert.level == 'critical' else '⚠️'
            print(f"  {icon} [{alert.level.upper()}] {alert.message}")
    else:
        print("  ✓ No alerts")

## Part 6: Retraining Decision

In [ ]:
class RetrainingDecision:
    """Decide when to retrain the model."""
    
    def __init__(self, config: Dict):
        self.config = config
        self.last_retrain = datetime.now() - timedelta(days=30)
    
    def should_retrain(self, metrics: Dict) -> Tuple[bool, str]:
        """Decide if model should be retrained."""
        
        # Check accuracy
        if metrics.get('accuracy_drop', 0) > self.config['accuracy_threshold']:
            return True, f"Accuracy dropped by {metrics['accuracy_drop']:.1%}"
        
        # Check drift
        if metrics.get('psi', 0) > self.config['psi_threshold']:
            return True, f"Significant drift detected (PSI: {metrics['psi']:.2f})"
        
        # Check time since last retrain
        days_since = (datetime.now() - self.last_retrain).days
        if days_since > self.config['max_days']:
            return True, f"Scheduled retrain ({days_since} days since last)"
        
        return False, "Model is healthy"

# Configure retraining
retrain_config = {
    'accuracy_threshold': 0.05,
    'psi_threshold': 0.25,
    'max_days': 30
}

retrain_decision = RetrainingDecision(retrain_config)

# Test decisions
print("=== Retraining Decisions ===")

for name, data in scenarios:
    perf = monitor_performance(model, data, features, baseline_metrics)
    drift = detect_drift(train_data, data, features)
    avg_psi = np.mean([r['psi'] for r in drift.values()])
    
    metrics = {
        'accuracy_drop': perf['degradation']['accuracy_drop'],
        'psi': avg_psi
    }
    
    should_retrain, reason = retrain_decision.should_retrain(metrics)
    
    print(f"\n{name}:")
    print(f"  Should retrain: {'YES' if should_retrain else 'No'}")
    print(f"  Reason: {reason}")

## Part 7: Visualization

In [ ]:
# Simulate monitoring over time
def simulate_production(model, baseline, days=30):
    """Simulate production monitoring over time."""
    metrics_over_time = []
    
    for day in range(days):
        # Gradually increase drift
        drift_factor = min(day / 30, 1.0) * 0.5
        concept_drift = max(0, (day - 15) / 30) * 0.5
        
        # Generate daily data
        daily_data = generate_data(200, drift_factor=drift_factor, concept_drift=concept_drift)
        
        # Calculate metrics
        X = daily_data[features]
        y = daily_data['churned']
        preds = model.predict(X)
        probs = model.predict_proba(X)[:, 1]
        
        accuracy = accuracy_score(y, preds)
        drift_results = detect_drift(train_data, daily_data, features)
        avg_psi = np.mean([r['psi'] for r in drift_results.values()])
        
        metrics_over_time.append({
            'day': day,
            'accuracy': accuracy,
            'psi': avg_psi,
            'pred_mean': probs.mean()
        })
    
    return pd.DataFrame(metrics_over_time)

# Run simulation
metrics_df = simulate_production(model, baseline_metrics, days=30)

# Plot
fig, axes = plt.subplots(3, 1, figsize=(12, 10))

# Accuracy over time
axes[0].plot(metrics_df['day'], metrics_df['accuracy'], 'b-', linewidth=2)
axes[0].axhline(y=baseline_metrics['accuracy'], color='g', linestyle='--', label='Baseline')
axes[0].axhline(y=baseline_metrics['accuracy'] - 0.05, color='r', linestyle='--', label='Critical threshold')
axes[0].fill_between(metrics_df['day'], 0, metrics_df['accuracy'], 
                     where=metrics_df['accuracy'] < baseline_metrics['accuracy'] - 0.05, 
                     alpha=0.3, color='red')
axes[0].set_ylabel('Accuracy')
axes[0].set_title('Model Accuracy Over Time')
axes[0].legend()
axes[0].set_ylim(0.5, 1.0)

# PSI over time
colors = ['green' if psi < 0.1 else 'orange' if psi < 0.25 else 'red' for psi in metrics_df['psi']]
axes[1].bar(metrics_df['day'], metrics_df['psi'], color=colors)
axes[1].axhline(y=0.1, color='orange', linestyle='--', label='Warning')
axes[1].axhline(y=0.25, color='red', linestyle='--', label='Critical')
axes[1].set_ylabel('PSI')
axes[1].set_title('Feature Drift (PSI) Over Time')
axes[1].legend()

# Prediction distribution
axes[2].plot(metrics_df['day'], metrics_df['pred_mean'], 'purple', linewidth=2)
axes[2].axhline(y=baseline_metrics['pred_mean'], color='g', linestyle='--', label='Baseline')
axes[2].set_xlabel('Day')
axes[2].set_ylabel('Mean Prediction')
axes[2].set_title('Prediction Distribution Shift')
axes[2].legend()

plt.tight_layout()
plt.show()

## Part 8: TODO - Build Your Monitoring Dashboard

Extend the monitoring system.

In [ ]:
# TODO: Add these to the monitoring system:
# 1. Latency monitoring
# 2. Error rate tracking
# 3. Custom business metrics
# 4. Automated report generation

print("Extend the monitoring system with additional metrics!")

## Self-Check

Run the cell below to verify your monitoring dashboard components are correct.

In [ ]:
# SELF-CHECK: Verify your monitoring setup

assert callable(calculate_psi), "calculate_psi function should exist"
assert callable(detect_drift), "detect_drift function should exist"
assert hasattr(alerting, 'alerts'), "AlertingSystem should track alerts"
assert hasattr(retrain_decision, 'should_retrain'), "RetrainingDecision should have decision method"
print(f"✅ Self-check passed! Monitoring system configured with {len(alerting.alerts)} alerts triggered")

---

## Stakeholder Summary

### TODO: Write a 3-bullet summary (~100 words) for the PM

Template:
• **What is drift:** When real-world data changes vs training data. Like a weather model trained on summer data failing in winter.
• **How we catch it:** Monitor PSI (data shift) and accuracy. Alert at PSI > 0.1 (warning) or > 0.25 (retrain trigger).
• **Business impact:** Without monitoring, degradation is silent. By the time retention offers stop working, we've already lost customers.

## Key Takeaways

1. **Types of drift:** Data drift, concept drift, label drift
2. **Detection methods:** KS test, PSI, accuracy monitoring
3. **Alerting:** Set appropriate thresholds, avoid fatigue
4. **Retraining:** Trigger on drift, accuracy drop, or schedule
5. **Monitor everything:** Inputs, outputs, performance, business metrics

### Congratulations!

You've completed the full ML curriculum! You now understand:
- ML fundamentals and problem framing
- Supervised learning algorithms
- Feature engineering and evaluation
- Deep learning and transformers
- LLMs and agentic AI
- MLOps and production monitoring

### Next Steps
- Complete the Capstone Project
- Apply these skills to real problems
- Keep learning and practicing!